### Thais Santos Barbosa - Tech Challenge Fase 2 - RM 357883

## Criando um Roteiro de Viagem

O problema de otimização escolhido foi maximizar a satisfação de um turista ao visitar determinadas atrações turísticas da cidade de São Paulo, respeitando um limite de tempo. Cada atração turística possui tempo de duração da visita e nível de interesse distintos.

Este problema envolve a alocação de recursos para visitar as atrações mais interessantes possíveis para o turista dentro do tempo máximo disponível para visitar atrações.

### Definição do Problema

- **Objetivo:** Maximizar a satisfação do turista.
- **Restrições:** O tempo total das visitas não pode exceder o tempo disponível.
- **Critérios de Sucesso:** Maximizar a satisfação do turista, garantindo que o tempo total das visitas não exceda o limite.
- **Recursos:**
  - **Tempo:** Cada turista possui um tempo máximo disponível para visitar as atrações.
  - **Atrações Turísticas:** Cada atração tem um tempo de visita e um nível de interesse associado.

O problema descrito é semelhante ao clássico problema da mochila, uma vez que o turista precisa selecionar as atrações (itens) de modo a maximizar sua satisfação (valor), sem ultrapassar o tempo disponível (capacidade da mochila).

A seguir, definimos os dados das atrações disponíveis para visita, onde cada tupla é organizada na ordem: atração, tempo, interesse.
Além disso, é definido o tempo máximo disponível para visitas.

In [127]:
ATRACOES = [
    ("Avenida Paulista", 2, 8),
    ("Parque do Ibirapuera", 3, 9),
    ("Museu de Arte de São Paulo (MASP)", 2, 10),
    ("Mercadão de São Paulo", 1.5, 7),
    ("Pinacoteca do Estado", 2, 9),
    ("Teatro Municipal", 1.5, 8),
    ("Estádio do Pacaembu", 2, 6),
    ("Beco do Batman", 1, 7),
    ("Jardim Botânico", 2, 8),
    ("Museu do Futebol", 2, 9)
]
TEMPO_DISPONIVEL = 10

Através da função **criar_individuo**, cada indivíduo será gerado como um vetor em que o número de elementos será igual ao número de atrações disponíveis.

A codificação utilizada foi a **codificação binária**. Sendo assim, cada elemento do vetor representa um gene e cada gene será 0 ou 1, dependendo de uma escolha aleatória. Se o gene for igual a 0, significa que a atração localizada no mesmo índice do vetor ATRACOES não foi escolhida para visita; se for igual a 1, significa que a atração foi escolhida.

A função seguinte, **criar_populacao**, utiliza a função criar_individuo para criar um determinado número de indivíduos e **inicializar a população de maneira aleatória**.

In [128]:
import random

def criar_individuo():
    return [random.randint(0, 1) for _ in range(len(ATRACOES))]

def criar_populacao(tamanho):
    return [criar_individuo() for _ in range(tamanho)]

A função **calcula_fitness_individuo** recebe um indivíduo da população e calcula o seu fitness. 

O fitness, aqui, é calculado com base no nível de interesse de cada atração. Portanto, o fitness de cada indivíduo será a soma dos níveis de interesse das atrações selecionadas para visita.

Os indivíduos cuja soma do tempo total de visita de cada atração escolhida ultrapassar o tempo disponível serão penalizados e retornarão fitness igual a 0, de modo a respeitar a restrição do tempo máximo disponível.

In [129]:
def calcula_fitness_individuo(individuo):
    total_tempo = 0
    total_interesse = 0

    for gene, atracao in zip(individuo, ATRACOES):
        if gene == 1:
            total_tempo += atracao[1]
            total_interesse += atracao[2]

    #print(total_tempo, "|", total_interesse)
    if total_tempo > TEMPO_DISPONIVEL:
        return 0

    return total_interesse

A função a seguir, **selecao_torneio_maior_fitness**, seleciona aleatoriamente um determinado número de indivíduos da população, ordena e retorna aquele que possuir maior fitness.

In [130]:
def selecao_torneio_maior_fitness(populacao_tor, tamanho_torneio):
    torneio = random.sample(populacao_tor, tamanho_torneio)
    torneio.sort(key=calcula_fitness_individuo, reverse=True)
    return torneio[0]

A função **cruzamento** recebe dois indivíduos pais e retorna dois indivíduos filhos. 

A **técnica de cruzamento utilizada foi a Two-Point Crossover**, em que dois pontos de cruzamento, são selecionados aleatoriamente dentro do comprimento dos pais, sendo possível dividir o indivíduo em três partes. A seguir, a parte do meio de um pai é substituída pela parte central do outro pai, combinando, assim, características de ambos os indivíduos.

A próxima função, **mutacao**, inverte ou não cada gene do indivíduo de acordo com a probabilidade descrita pela taxa de mutação.

In [131]:
def cruzamento(pai1, pai2):
    ponto1 = random.randint(0, round(len(pai1) / 2))
    ponto2 = random.randint(ponto1 + 1, len(pai1))
    
    filho1 = pai1[:ponto1] + pai2[ponto1:ponto2] + pai1[ponto2:]
    filho2 = pai2[:ponto1] + pai1[ponto1:ponto2] + pai2[ponto2:]
    
    return filho1, filho2

def mutacao(individuo, taxa_mutacao):
    for i in range(len(individuo)):
        if random.random() < taxa_mutacao:
            individuo[i] = 1 - individuo[i]

Para implementação do algoritmo genético, algumas variáveis serão inicializadas com valores fixos, sendo elas:
- **Número de gerações/populações**;
- **Quantidade de indivíduos por geração/população**;
- **Taxa de cruzamento** => define se haverá cruzamento para gerar dois novos indivíduos a partir de dois indivíduos pais ou se serão mantidos os indivíduos pais;
- **Taxa de mutação** => define em qual proporção um indivíduo poderá sofrer mutação;
- **Tamanho do torneio** => define qual número de indivíduos será escolhido aleatoriamente dentre a população para sofrer seleção natural pelo maior fitness;
- **Elitismo** => define quantos dos melhores indivíduos, sendo aqueles que possuem maior fitness, serão mantidos na nova geração/população. Se for igual a zero, o melhor indivíduo não será preservado.
- **Quantidade de gerações para conversão ao melhor fitness** => define quantos rodadas sem mudança do resultado de melhor fitness são necessárias para afirmar que o algoritmo já convergiu para a solução subótima.

Os valores destas variáveis foram escolhidos conforme desempenho nos testes realizados. Portanto, os valores apresentados acima são aqueles que resultaram numa melhor convergência do algoritmo à uma solução subótima aceitável.

In [132]:
n_geracoes = 20
tamanho_populacao = 20
taxa_cruzamento = 0.9
taxa_mutacao = 0.8
tamanho_torneio = 2
elitismo = 0
qtd_geracoes_melhor_fitness = 4

### Implementação do Algoritmo Genético

A seguir, o algoritmo genético, que utilizará todas as funções descritas acima é implementado. 

#### 1 - População Inicial
Primeiramente, são inicializados de forma aleatória todos os indivíduos que formarão a primeira população. 

#### 2 - Formação de Novas Gerações
Em seguida, ocorre a criação de cada geração. 

##### 2.1 - Elitismo
Ocorre quando cada uma das gerações é inicializada com o indivíduo de melhor fitness da geração anterior. 

##### 2.2 - Cruzamento
Os demais indivíduos desta geração são criados a partir do cruzamento de diferentes pares de indivíduos pais da população anterior, sendo que alguns destes indíviduos podem ser mantidos na nova geração, dada a taxa de cruzamento.

##### 2.3 - Seleção Natural
Os indivíduos pais escolhidos para realizar o cruzamento, são aqueles de melhor fitness, selecionados, utilizando a técnica de torneio, dentre grupos de três indivíduos da geração anterior, formados aleatoriamente.

##### 2.4 - Mutação
Então, os dois indivíduos filhos gerados sofrem mutação, pela técnica de bit-flip. Ou seja, na proporção dada pela taxa de mutação, cada indivíduo sofrerá a inversão dos valores de um ou mais de seus bits, selecionados aleatoriamente.

Após a mutação, os indivíduos filhos passam a compor a nova geração e novos filhos são gerados até que cada geração alcance seu tamanho máximo.

#### 3 - Convergência para Solução Subótima
O processo é repetido até a formação da última geração, cujo indivíduo de melhor fitness será a solução subótima, que representa uma aceitável alocação de recursos para visitar as atrações mais interessantes possíveis para o turista dentro do tempo máximo disponível para visita.

In [133]:
import time

start_time = time.perf_counter()

populacao = criar_populacao(tamanho_populacao)

ja_informou_convergencia = False
melhores_fitness_geracoes = []
geracao_convergencia = -1
for geracao in range(n_geracoes):
    # Preserva os melhores indivíduos (elitismo)
    populacao_ordenada = sorted(populacao, key=calcula_fitness_individuo, reverse=True)
    melhor_ind_geracao = populacao_ordenada.copy()[:elitismo]
    nova_populacao = []
    
    # Preenche o resto da população com cruzamento e mutação
    while len(nova_populacao) < (tamanho_populacao - elitismo):
        pai1 = selecao_torneio_maior_fitness(populacao, tamanho_torneio)
        pai2 = selecao_torneio_maior_fitness(populacao, tamanho_torneio)
        
        if random.random() < taxa_cruzamento:
            filho1, filho2 = cruzamento(pai1, pai2)
        else:
            filho1, filho2 = pai1, pai2
        
        mutacao(filho1.copy(), taxa_mutacao)
        mutacao(filho2.copy(), taxa_mutacao)
        
        nova_populacao.append(filho1)
        nova_populacao.append(filho2)

    nova_populacao.extend(melhor_ind_geracao.copy())
    
    populacao = nova_populacao.copy()
    nova_populacao = []

    # Calcular fitness para cada indivíduo da nova população
    fitness_populacao = [calcula_fitness_individuo(individuo) for individuo in populacao]
    print(f"Fitness dos indivíduos da geração {geracao} = {fitness_populacao}")
    
    melhor_individuo = max(populacao, key=calcula_fitness_individuo)
    melhor_fitness = calcula_fitness_individuo(melhor_individuo)
    print(f"Melhor fitness desta geração = {melhor_fitness}\n")
    melhores_fitness_geracoes.append(melhor_fitness)
    
    if not ja_informou_convergencia:
        convergiu_qtd_geracoes = True
        if geracao >= qtd_geracoes_melhor_fitness:
            for i in range(geracao, geracao - qtd_geracoes_melhor_fitness, -1):
                if melhores_fitness_geracoes[geracao] != melhores_fitness_geracoes[i]:
                    convergiu_qtd_geracoes = False
                    break
            if convergiu_qtd_geracoes and not ja_informou_convergencia:
                geracao_convergencia = geracao
                ja_informou_convergencia = True

end_time = time.perf_counter()
elapsed_time = end_time - start_time

print(f"\nAlgoritmo Genético")
print(f"Melhor indivíduo encontrado: {melhor_individuo} - Fitness: {melhor_fitness}")
print(f"Tempo gasto: {elapsed_time:.6f} segundos")
print(f"Convergiu na geração: {geracao_convergencia}")

Fitness dos indivíduos da geração 0 = [32, 23, 41, 33, 25, 32, 30, 34, 25, 33, 39, 33, 26, 39, 34, 0, 23, 0, 33, 32]
Melhor fitness desta geração = 41

Fitness dos indivíduos da geração 1 = [0, 24, 32, 39, 40, 25, 24, 0, 15, 0, 47, 33, 41, 30, 38, 34, 40, 25, 32, 33]
Melhor fitness desta geração = 47

Fitness dos indivíduos da geração 2 = [41, 39, 0, 16, 32, 0, 30, 41, 15, 40, 32, 41, 38, 33, 25, 42, 0, 32, 0, 38]
Melhor fitness desta geração = 42

Fitness dos indivíduos da geração 3 = [39, 40, 32, 39, 25, 0, 41, 38, 0, 23, 42, 24, 23, 33, 40, 39, 30, 47, 15, 47]
Melhor fitness desta geração = 47

Fitness dos indivíduos da geração 4 = [40, 40, 42, 38, 34, 47, 30, 26, 33, 41, 39, 31, 30, 49, 41, 24, 47, 47, 40, 47]
Melhor fitness desta geração = 49

Fitness dos indivíduos da geração 5 = [47, 40, 30, 40, 40, 33, 47, 47, 40, 40, 0, 15, 0, 39, 40, 0, 38, 0, 48, 26]
Melhor fitness desta geração = 48

Fitness dos indivíduos da geração 6 = [31, 23, 0, 31, 47, 40, 47, 38, 0, 47, 33, 47, 47, 33

### Testes de Eficácia

Com o fim de comprovar a eficácia do algoritmo genético implementado acima, os resultados obtidos serão comparados, a seguir, com os resultados de dois métodos convencionais para resolver problemas de alocação de recursos, tais quais o clássico problema da mochila e este, apresentado aqui, da elaboração de um roteiro de viagem.

Os dois métodos convencionais escolhidos foram Algoritmo Guloso (Greedy) e Programação Dinâmica.

#### Algoritmo Guloso (Greedy)

Seleciona as atrações com maior valor (nível de interesse) até preencher o limite de tempo. Apesar de não garantir a solução ótima, suas principais vantagens são a velocidade de execução e o baixo uso de recursos computacionais.

#### Programação Dinâmica

Testa todas as combinações possíveis de atrações, respeitando a restrição de tempo. Este método garante a solução ótima, porém exige mais recursos computacionais e possui implementação mais complexa.

In [134]:
# Implementação do algoritmo greedy

start_time = time.perf_counter()

atracoes_para_filtrar_maior = ATRACOES.copy()

individuo_gerado_algoritmo_greedy = [0 for _ in range(len(atracoes_para_filtrar_maior))]
tempo_individuo_gerado_algoritmo_greedy = 0

while tempo_individuo_gerado_algoritmo_greedy < TEMPO_DISPONIVEL:
    par_maior_interesse = max(enumerate(atracoes_para_filtrar_maior), key=lambda x: x[1][2])
    indice_maior_interesse = par_maior_interesse[0]
    atracao_maior_interesse = par_maior_interesse[1]

    if (tempo_individuo_gerado_algoritmo_greedy + atracao_maior_interesse[1]) <= TEMPO_DISPONIVEL: 
        individuo_gerado_algoritmo_greedy[indice_maior_interesse] = 1
        tempo_individuo_gerado_algoritmo_greedy += atracao_maior_interesse[1]
        
    atracoes_para_filtrar_maior[indice_maior_interesse] = (
        atracao_maior_interesse[0], 
        atracao_maior_interesse[1], 
        0  # Interesse agora é 0
    )
    
end_time = time.perf_counter()
elapsed_time = end_time - start_time

print(f"Algoritmo Greedy")
print(f"Melhor indivíduo encontrado: {individuo_gerado_algoritmo_greedy} - Fitness: {calcula_fitness_individuo(individuo_gerado_algoritmo_greedy)}")
print(f"Tempo gasto: {elapsed_time:.6f} segundos")

Algoritmo Greedy
Melhor indivíduo encontrado: [0, 1, 1, 0, 1, 0, 0, 1, 0, 1] - Fitness: 44
Tempo gasto: 0.000470 segundos


In [135]:
# Implementação da programação dinâmica

from itertools import product

start_time = time.perf_counter()

# Gera todas as combinações possíveis de vetores de 10 posições com 0 e 1
todas_as_combinacoes_possiveis = list(product([0, 1], repeat=len(ATRACOES)))

individuo_gerado_algoritmo_pd = max(todas_as_combinacoes_possiveis, key=calcula_fitness_individuo)
melhor_fitness = calcula_fitness_individuo(individuo_gerado_algoritmo_pd)

end_time = time.perf_counter()
elapsed_time = end_time - start_time

print(f"Algoritmo Programação Dinâmica")
print(f"Melhor indivíduo encontrado: {individuo_gerado_algoritmo_pd} - Fitness: {calcula_fitness_individuo(individuo_gerado_algoritmo_pd)}")
print(f"Tempo gasto: {elapsed_time:.6f} segundos")

Algoritmo Programação Dinâmica
Melhor indivíduo encontrado: (0, 0, 1, 1, 1, 1, 0, 1, 0, 1) - Fitness: 50
Tempo gasto: 0.001342 segundos


### Análise dos Resultados

Os resultados obtidos para o problema de maximização da satisfação do turista em visitas a atrações turísticas revelam um desempenho variável entre os algoritmos Genético, Greedy e Programação Dinâmica. Após sucessivas execuções, observa-se que o algortimo genético pode, ocasionalmente, encontrar soluções melhores do que as obtidas por métodos convencionais.

#### Comparação de Desempenho

##### Qualidade da Solução

Neste aspecto, a Programação Dinâmica é a mais eficaz, uma vez que fornece a solução ótima para o problema. A natureza determinística deste algoritmo permite uma busca exaustiva, garantindo a melhor solução possível.
O Algoritmo Greedy mostrou-se uma escolha rápida e eficiente, mas com a limitação de não considerar todas as combinações possíveis de atrações, o que pode resultar em soluções subótimas.
O Algoritmo Genético, por sua vez, foi capaz de alcançar resultados iguais aos da programação dinâmica e soluções frequentemente melhores do que as oferecidas pelo Algoritmo Greedy.

##### Tempo de Execução

Em termos de tempo de execução, o Algoritmo Greedy foi o mais rápido, mostrando sua eficiência em encontrar soluções rapidamente, mas não necessariamente as melhores.
A Programação Dinâmica apresenta, neste caso, tempos de execução competitivos para encontrar a solução ótima, o que é esperado para problemas de tamanho e complexidade moderados, como o aqui apresentado.
O Algoritmo Genético frequentemente apresentou tempo de execução maior que os demais métodos devido à sua natureza de exploração e iterações múltiplas.

#### Vantagens de Utilizar o Algoritmo Genético

##### Variabilidade

O Algoritmo Genético é projetado para explorar o espaço de soluções de maneira aleatória, o que significa que pode, em algumas execuções, encontrar combinações de atrações que superem as soluções fornecidas por métodos convencionais como o Greedy. Essa variabilidade pode ser vantajosa em certos contextos.

##### Performance em Grande Escala

Para problemas de otimização de alta dimensão ou com um espaço de busca complexo, os algoritmos genéticos podem ser mais eficazes do que abordagens exatas, já que estas últimas podem ser inviáveis devido à limitações computacionais. Assim, o Algoritmo Genético se destaca como uma alternativa quando a garantia da solução ideal se torna impraticável.

#### Conclusão

O Algoritmo Genético tem potencial para oferecer soluções melhores que métodos mais determinísticos em problemas mais complexos. A escolha entre esses algoritmos deve considerar fatores como a natureza do problema, os recursos computacionais disponíveis, o tempo disponível para a execução e a necessidade de garantia de uma solução ótima versus uma solução suficientemente boa e de rápida execução. 